<a href="https://colab.research.google.com/github/srirambandi/ai/blob/master/examples/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn checkout https://github.com/srirambandi/ai/trunk/examples/MNIST

In [0]:
import ai
import numpy as np

In [0]:
def load_data(file):
    dict = np.load(file, allow_pickle=True)
    return dict

In [0]:
train_file = 'MNIST/train.npy'
test_file = 'MNIST/test.npy'

In [0]:
ai.manual_seed(2357)

In [0]:
class MLP(ai.Module):
    def __init__(self, ):
        self.fc1 = ai.Linear(784, 200)
        self.fc2 = ai.Linear(200, 10)
        
    def forward(self, x):
        o1 = ai.G.dropout(ai.G.relu(self.fc1.forward(x)), p=0.75)
        o2 = ai.G.softmax(self.fc2.forward(o1))
        return o2

In [20]:
mlp = MLP()
print(mlp)

MLP(
  fc1: Linear(input_features=784, output_features=200, bias=True)
  fc2: Linear(input_features=200, output_features=10, bias=True)
)


In [0]:
L = ai.Loss(loss_fn='CrossEntropyLoss')
optim = ai.Optimizer(mlp.parameters(), optim_fn='Adam', lr=1e-3)

In [0]:
train_dict = load_data(train_file)
inputs = train_dict.item()['data']
outputs = train_dict.item()['labels']
del train_dict

In [0]:
it, epoch = 0, 0
m = 128

In [0]:
def evaluate():
    ai.G.grad_mode = False
    file = test_file
    dict = load_data(file)
    inputs = dict.item()['data']
    outputs = dict.item()['labels']
    correct, total = 0, 0
    test_m = m
    for batch in range(int(len(outputs) / m)):
        input = inputs[batch * test_m : (batch + 1) * test_m] / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = np.array(outputs[batch * test_m : (batch + 1) * test_m])
        scores = mlp.forward(input)
        preds = np.argmax(scores.data, axis=0)
        correct += np.sum(np.equal(output, preds))
        total += test_m
    accuracy = float(correct / total)
    ai.G.grad_mode = True
    return accuracy

In [25]:
while epoch < 15:
    epoch += 1
    it = 0
    for batch in range(int(len(outputs) / m)):
    # for batch in range(1):
        input = inputs[batch * m : (batch + 1) * m] / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = outputs[batch * m : (batch + 1) * m]
        onehot = np.zeros((10, m))
        for _ in range(m):
            onehot[output[_], _] = 1.0
        scores = mlp.forward(input)
        loss = L.loss(scores, onehot)
        loss.backward()
        optim.step()        # update parameters with optimization functions
        optim.zero_grad()   # clearing the backprop list and resetting the gradients to zero
        if it%50 == 0:
            print('epoch: {}, iter: {}, loss: {}'.format(epoch, it, loss.data[0, 0]))
        it += 1
    print('Epoch {} completed. Accuracy: {:.2%} \n'.format(epoch, evaluate()))

using Adam
epoch: 1, iter: 0, loss: 2.3029468194362273
epoch: 1, iter: 50, loss: 0.744773933515146
epoch: 1, iter: 100, loss: 0.4461368333963829
epoch: 1, iter: 150, loss: 0.3924592712027108
epoch: 1, iter: 200, loss: 0.3276155178169168
epoch: 1, iter: 250, loss: 0.34157984271946407
epoch: 1, iter: 300, loss: 0.27599564051208175
epoch: 1, iter: 350, loss: 0.42015027652099113
epoch: 1, iter: 400, loss: 0.332420048816242
epoch: 1, iter: 450, loss: 0.32262224911312576
Epoch 1 completed. Accuracy: 92.83% 

epoch: 2, iter: 0, loss: 0.22569922187600583
epoch: 2, iter: 50, loss: 0.23741324849821216
epoch: 2, iter: 100, loss: 0.16844311398976142
epoch: 2, iter: 150, loss: 0.21456708947819647
epoch: 2, iter: 200, loss: 0.17467302930789977
epoch: 2, iter: 250, loss: 0.2273470363272249
epoch: 2, iter: 300, loss: 0.1835680741701417
epoch: 2, iter: 350, loss: 0.3081284886653894
epoch: 2, iter: 400, loss: 0.22054432726155443
epoch: 2, iter: 450, loss: 0.2531417764028556
Epoch 2 completed. Accuracy: 

In [26]:
mlp.save()

saving model...
Successfully saved model in MLP.npy
